## Import packages

In [1]:
!pip install transformers --upgrade
!pip install colpali-engine --upgrade
!pip install qdrant-client[fastembed]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 81.5 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 87.2 MB/s eta 0:00:00:00:010:01
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=95ebee3432e51ab49616b88f3d56aa905f88293c0e81b11f3464a04d9ae88baa
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil
  Attempting uninstall: transformers
    Found existing installation: transformers

In [2]:
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.4 MB/s eta 0:00:000:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.10
    Uninstalling google-ai-generativelanguage-0.6.10:
      Successfully uninstalled google-ai-generativelanguage-0.6.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependen

In [3]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler118
The following NEW packages will be installed:
  poppler-utils
The following packages will be upgraded:
  libpoppler-dev libpoppler-private-dev libpoppler118
3 upgraded, 1 newly installed, 0 to remove and 126 not upgraded.
Need to get 1,462 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-private-dev amd64 22.02.0-2ubuntu0.6 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-dev amd64 22.02.0-2ubuntu0.6 [5,184 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.6 [1,071 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 1,4

In [4]:
from colpali_engine.models import ColPali, ColPaliProcessor, ColQwen2, ColQwen2Processor
from datasets import load_dataset 
from qdrant_client import QdrantClient, models
import torch
from tqdm import tqdm
import uuid
import numpy as np
import random
from PIL import Image
import qdrant_utils

In [ ]:
from colpali_engine.models import ColPali, ColPaliProcessor
from qdrant_client import QdrantClient, models
from PIL import Image
from langchain_google_genai import ChatGoogleGenerativeAI
import qdrant_utils
import numpy as np
import torch
import uuid

## Qdrant and Gemini API

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Gemini_api")
secret_value_1 = user_secrets.get_secret("qdrant_api")

In [7]:
from qdrant_client import QdrantClient
client = QdrantClient( url='https://d3ceafa0-b886-47c4-aa87-c4e9c91905d6.eu-central-1-0.aws.cloud.qdrant.io',api_key = secret_value_1)
print(client.get_collections())

collections=[CollectionDescription(name='hinh_khong_gian'), CollectionDescription(name='hinhkhonggian')]


In [8]:
import google.generativeai as genai
from PIL import Image
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=secret_value_0)

## Colpali Embedding

In [9]:
colpali_model = ColPali.from_pretrained(
        "vidore/colpali-v1.3",
        torch_dtype=torch.bfloat16,
        device_map="cuda:0",  # Use "cuda:0" for GPU, "cpu" for CPU, or "mps" for Apple Silicon
    ).eval()

colpali_processor = ColPaliProcessor.from_pretrained("vidore/colpali-v1.3")

adapter_config.json:   0%|          | 0.00/751 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/78.6M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.6M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

## Embedding queries image and retrieve

In [10]:
# query = "Góc giữa đường thẳng và mặt phẳng"

# colpali_query = qdrant_utils.batch_embed_query([query], colpali_processor, colpali_model)


query_path = '/kaggle/input/hinh-khong-gian-images/pdf_hinh_khong_gian/page_2.png'
query_image = Image.open(query_path).convert('RGB')

colpali_query = qdrant_utils.batch_embed_query_image([query_image], colpali_processor, colpali_model)

In [13]:
collection_name = 'hinhkhonggian'
answer_colpali = qdrant_utils.reranking_search_batch(client,colpali_query, collection_name)
retrieve_path = answer_colpali[0].points[0].payload['source']

In [14]:
retrieve_path

'/kaggle/working/images/page_2.jpg'

## LLM model

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage
import base64

with open(retrieve_path, "rb") as f:
    img_bytes = f.read()
query_img_base64 = base64.b64encode(img_bytes).decode("utf-8")
query_img_data_uri = f"data:image/png;base64,{query_img_base64}"

with open("/kaggle/input/unsolving-math-problem/unsolving_problem/1.png", "rb") as f:
    img_bytes = f.read()
question_img_base64 = base64.b64encode(img_bytes).decode("utf-8")
question_img_data_uri = f"data:image/png;base64,{question_img_base64}"

messages = [
    SystemMessage(
        content=f"You are an helpful mathematical chatbot. User will provide you 2 image, the first one is relevant problem, you can base on that to solve the second one. Give user the correct answer with clear step by step solution."
    ),
    HumanMessage(
        content= [
                {
                    "type": "text",
                    "text": "Solve for me this problem, step by step!"
                },
                {
                    "type":"image_url",
                    "image_url": query_img_data_uri
                },   
                {
                    "type":"image_url",
                    "image_url": question_img_data_uri
                }, 
        ]
    )
]

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/images/page_2.jpg'

In [ ]:
llm.invoke(messages)